## PROJETO - AUTOMAÇÃO NO PROCESSO DE REPORT DIÁRIO

### DESCRIÇÃO:
- O projeto consiste em criar um processo de automação de envio de e-mails. 
- Será calculado o OnePage de cada loja individualmente e enviaremos um e-mail para o gerente de cada loja com o seu OnePage no corpo do e-mail e também o arquivo completo com os dados da sua respectiva loja em anexo.
- Ao final, será enviado um e-mail para a Diretoria com o ranking de faturamento anual e o ranking faturamento do último dia, ressaltando no corpo do e-mail qual foi a melhor e a pior loja.

### ETAPAS DO PROCESSO:
- Importar e tratar as bases de dados
- Criar 1 arquivo para cada loja
- Salvar o backup nas pastas
- Calcular os indicadores
- Enviar o OnePage
- Enviar o e-mail para diretoria

#### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
# Importar bibliotecas:
import pandas as pd
import pathlib
import yagmail
import win32com.client as win32
from tqdm import tqdm

In [10]:
# Importar base de dados: 
emails = pd.read_excel(r'Bases de dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

In [11]:
display(emails)

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,Iguatemi_Esplanada+helena@gmail.com
1,Shopping Midway Mall,Alice,Shopping_Midway_Mall+alice@gmail.com
2,Norte Shopping,Laura,Norte_Shopping+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,Shopping_Iguatemi_Fortaleza+manuela@gmail.com
4,Shopping União de Osasco,Valentina,Shopping_União_de_Osasco+valentina@gmail.com
5,Shopping Center Interlagos,Sophia,Shopping_Center_Interlagos+sophia@gmail.com
6,Rio Mar Recife,Isabella,Rio_Mar_Recife+isabella@gmail.com
7,Salvador Shopping,Heloisa,Salvador_Shopping+heloisa@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,Rio_Mar_Shopping_Fortaleza+luiza@gmail.com
9,Shopping Center Leste Aricanduva,Julia,Shopping_Center_Leste_Aricanduva+julia@gmail.com


#### Passo 2 - Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [4]:
# Incluir nome da Loja em vendas:
vendas = vendas.merge(lojas, on='ID Loja')

# Criar um dicionário com o nome da loja como chave e os dados da tabela referente a essa loja como os valores:
dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]

# Defidindo o dia do indicador:
dia_indicador = vendas['Data'].max()

#### Passo 3 - Salvar a planilha na pasta de backup

In [5]:
# Identificar se a pasta da loja já existe:
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in tqdm(dicionario_lojas):
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    
# Salvar o arquivo dentro da pasta da loja:
    nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:37<00:00,  1.52s/it]


#### Passo 4 - Definindo as metas diárias e anuais:

In [6]:
# Criação das variáveis com as metas:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

#### Passo 5 - Calcular os indicadores e enviar os e-mails para os gerentes de cada loja

In [7]:
# Criando um loop que irá enviar os e-mails automáticamente:
for loja in tqdm(dicionario_lojas):
    
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    #Calculo do faturamento:
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()
    
    #Calculo da diversidade de produtos:
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
    
    #Calculo do ticket medio: 
    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    
    # Enviar e-mail:
    ## Definindo variáveis para o envio do e-mail automatizado: 
    email_destinatario = emails.loc[emails['Loja']==loja,'E-mail'].values[0]
    email_destinatario_name = emails.loc[emails['Loja']==loja,'Gerente'].values[0]
    email_subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'
    email_anexo = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'

    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'

    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'

    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'

    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'

    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    email_bory = f'''
    <p> Bom dia, {email_destinatario_name} </p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da loja <strong>{loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R$ {faturamento_dia:.2f}</td>
        <td style="text-align: center">R$ {meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R$ {ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R$ {meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>

    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R$ {faturamento_ano:.2f}</td>
        <td style="text-align: center">R$ {meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R$ {ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R$ {meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>

    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes. </p>

    <p>Qualquer dúvida estou à disposição. </p>

    <p>Atenciosamente,</p>
    <p>Equipe de Inteligência de Mercado</p>
    '''
# ################# Enviando com GMAIL #################
#     # Definindo as credenciais de acesso("usuário" e "senha app") ao gmail: 
#     usuario = yagmail.SMTP(user='digite seu email aqui', password='digite sua Senha APP aqui')

#     # Enviando o e-mail:
#     usuario.send(to=email_destinatario,
#                  subject=email_subject,
#                  contents=email_bory,
#                  attachments=email_anexo)
    
################# Enviando com Outlook #################
    # Definindo as variáveis do outlook:
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    nome = email_destinatario_name
    mail.To = email_destinatario
    mail.Subject = email_subject
    mail.HTMLBody = email_bory

    # Anexando arquivo: 
    attachment = email_anexo
    mail.attachments.Add(str(attachment))

    # Enviando o e-mail: 
    mail.Send()

print('E-mail das Lojas enviados.')

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [02:03<00:00,  4.95s/it]

E-mail das Lojas enviados.


##### Passo 6 - Criar Ranking para a Diretoria

In [8]:
# Agrupando e ordenando o total geral de vendas por loja:
faturamento_lojas = vendas.groupby('Loja')[['Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
##Salvando o arquivo:
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

# Agrupando e ordenando as vendas do dia anterior por loja: 
vendas_dia = vendas.loc[vendas['Data']==dia_indicador,:]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
##Salvando o arquivo:
nome_arquivo = f'{dia_indicador.month}_{dia_indicador.day}_Ranking do Dia.xlsx'
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

#### Passo 7 - Enviar e-mail para Diretoria

In [9]:
# Enviar e-mail:
## Definindo variáveis para envio de e-mail automatizado: 
email_destinatario = emails.loc[emails['Loja']=='Diretoria','E-mail'].values[0]
email_destinatario_name = emails.loc[emails['Loja']=='Diretoria','Gerente'].values[0]
email_subject = f'Ranking do Dia - {dia_indicador.day}/{dia_indicador.month}'
email_anexo_1 = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx'
email_anexo_2 = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_Ranking do Dia.xlsx'
email_bory = f'''
Prezados, bom dia!

Melhor loja do dia em Faturamenteo: Loja {faturamento_lojas_dia.index[0]} com Faturamento R$ {faturamento_lojas_dia.iloc[0, 0]:.2f}.
Pior loja do dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R$ {faturamento_lojas_dia.iloc[-1, 0]:.2f}.

Melhor loja do ano em Faturamenteo: Loja {faturamento_lojas_ano.index[0]} com Faturamento R$ {faturamento_lojas_ano.iloc[0, 0]:.2f}.
Pior loja do ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com Faturamento R$ {faturamento_lojas_ano.iloc[-1, 0]:.2f}.


Segue em anexo os ranking do ano e do dia de todas as lojas.

Qualquer dúvida estou à disposição.

Atenciosamente,
Equipe de Inteligência de Mercado

''' 
    
# ################# Enviando com GMAIL #################
# # Definindo as credenciais de acesso("usuário" e "senha app") ao gmail: 
# usuario = yagmail.SMTP(user='digite seu email aqui', password='digite sua Senha APP aqui')

# # Enviando o e-mail:
# usuario.send(to=email_destinatario,
#              subject=email_subject,
#              contents=email_bory,
#              attachments=email_anexo)
    
################# Enviando com Outlook #################
# Definindo as variaveis do outlook:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
nome = email_destinatario_name
mail.To = email_destinatario
mail.Subject = email_subject
mail.Body = email_bory

# Anexando arquivo: 
attachment = email_anexo_1
mail.attachments.Add(str(attachment))
attachment = email_anexo_2
mail.attachments.Add(str(attachment))

# Enviando o e-mail: 
mail.Send()

print('E-mail da Diretoria enviado.')

E-mail da Diretoria enviado.
